In [1]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from connection_keys import connect_bq_client
import statsmodels.api as sm
import os
import warnings
import itertools
warnings.filterwarnings("ignore")

In [8]:
client = connect_bq_client()

sql = """
    SELECT *
    FROM `db_market_product.gpu`
    ORDER BY dt_carga asc
"""

df = client.query(sql).to_dataframe()
df['data'] = df.dt_carga.dt.date
df['ds_nome_produto'] = df['ds_nome_produto'].str.upper()

In [78]:
df.sample(10)

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data,ds_nome_produto_resumido
126015,EVGA,PLACA DE VIDEO EVGA GEFORCE GTX 970 4GB GDDR5 ...,1351.16,1351.16,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-19 22:05:16.945857+00:00,2022-10-19,GTX 970
149777,Asus,PLACA DE VÍDEO ASUS TUF NVIDIA RTX 3090 TI - 2...,15569.90,14791.41,5.0,0,None,False,Kabum,2022-10-24 10:03:21.452920+00:00,2022-10-24,RTX 3090 TI
151202,Galax,PLACA DE VIDEO GALAX GEFORCE RTX 3070 1-CLICK ...,5789.06,4431.81,12.0,<NA>,"[{'ds_oferta_nome': 'HalloPruu', 'vl_oferta_pr...",IN_STOCK,Pichau,2022-10-24 10:03:21.452920+00:00,2022-10-24,RTX 3070
118925,Asus,PLACA DE VIDEO ASUS RADEON RX 460 OC 4GB GDDR5...,594.60,594.60,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-18 10:01:50.109584+00:00,2022-10-18,RX 460
55810,Gigabyte,PLACA DE VÍDEO GIGABYTE AMD RADEON RX 6650 XT ...,3129.40,2659.99,15.0,0,"[{'ds_oferta_nome': 'Prime', 'vl_oferta_preco_...",False,Kabum,2022-10-09 11:23:24.394604+00:00,2022-10-09,RX 6650
247058,Duex,PLACA DE VIDEO DUEX GEFORCE GTX 750TI 4GB DDR5...,1179.20,1179.20,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-11 10:02:12.034969+00:00,2022-11-11,
133696,PCYES,PLACA DE VIDEO PCYES GEFORCE GTX 1660 TI 6GB G...,3260.00,2999.20,8.0,1,None,True,Kabum,2022-10-21 10:01:40.506674+00:00,2022-10-21,GTX 1660 TI
79474,MSI,PLACA DE VÍDEO MSI NVIDIA GEFORCE GTX 1650 D6 ...,1882.34,1599.99,15.0,20,"[{'ds_oferta_nome': 'Hora do Play', 'vl_oferta...",True,Kabum,2022-10-12 22:39:14.465550+00:00,2022-10-12,GTX 1650
319281,Afox,"PLACA DE VÍDEO RX 560 AFOX RADEON, 4GB DDR5, 1...",1528.22,1298.99,15.0,0,"[{'ds_oferta_nome': 'Prime', 'vl_oferta_preco_...",False,Kabum,2022-11-24 22:01:24.994917+00:00,2022-11-24,RX 560
123309,Zotac,PLACA DE VIDEO ZOTAC GEFORCE GTX 950 2GB GDDR5...,774.56,774.56,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-19 10:04:13.668321+00:00,2022-10-19,GTX 950


In [147]:
df.iloc[df.index == 187149]['ds_nome_produto'].tolist() # ds_nome_produto_resumido ds_nome_produto

['PLACA DE VÍDEO EVGA NVIDIA GEFORCE RTX 3060 XC BLACK GAMING 12G P5 3655 KR, LHR,  12GB GDDR6, DLSS, RAY TRACING - 12G-P5-3655-KR']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384100 entries, 0 to 384099
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   ds_marca                    384100 non-null  object             
 1   ds_nome_produto             384100 non-null  object             
 2   vl_preco_atual              384100 non-null  float64            
 3   vl_preco_com_desconto       384100 non-null  float64            
 4   vl_porcentagem_do_desconto  384100 non-null  float64            
 5   qt_quantidade_estoque       142715 non-null  Int64              
 6   oferta                      384100 non-null  object             
 7   ds_disponibilidade          384100 non-null  object             
 8   loja                        384100 non-null  object             
 9   dt_carga                    384100 non-null  datetime64[ns, UTC]
 10  data                        384100 non-null 

In [4]:
df.groupby(['dt_carga'])['loja'].count()
df.groupby(['data', 'loja'])['dt_carga'].nunique()

data        loja  
2022-09-29  Kabum     2
            Pichau    2
2022-09-30  Kabum     2
            Pichau    2
2022-10-01  Kabum     2
                     ..
2022-12-04  Pichau    2
2022-12-05  Kabum     2
            Pichau    2
2022-12-06  Kabum     1
            Pichau    1
Name: dt_carga, Length: 138, dtype: int64

In [5]:
# df_datas_loja = df.groupby(['data', 'loja'])['dt_carga'].nunique()
# df_datas_loja
maxDf = df.groupby(['data'])['dt_carga'].max()
minDf = df.groupby(['data'])['dt_carga'].min()
maxDates = maxDf.tolist()
df = df.loc[df['dt_carga'].isin(maxDates)]

In [45]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [76]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent
ex = "PLACA DE VÍDEO ASUS NVIDIA GEFORCE GTX 1050 TI OC CERBERUS, 4GB GDDR5 - CERBERUS-GTX1050TI-O4G"
sent = preprocess(ex)
sent
pattern = 'NP: {<NNP>*<CD>*<NNP>}'
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  (NP
    PLACA/NNP
    DE/NNP
    VÍDEO/NNP
    ASUS/NNP
    NVIDIA/NNP
    GEFORCE/NNP
    GTX/NNP
    1050/CD
    TI/NNP)
  (NP OC/NNP CERBERUS/NNP)
  ,/,
  (NP 4GB/CD GDDR5/NNP)
  -/:
  CERBERUS-GTX1050TI-O4G/NN)


In [57]:
def get_resume_product_name(name):
    # string = df['nome_produto'][df['id'] == 259].item()
    pattern = "(RTX|GTX|GT|RX|WX|Quadro|R7|R5|R9|HD)\s(\d+)"
    resume_name = re.search(pattern, name)
    if resume_name is None:
        return ''
    else:
        return resume_name.group()

df['ds_nome_produto_resumido'] = df.apply(lambda x: get_resume_product_name(x['ds_nome_produto']), axis=1)

In [139]:
def get_resume_product_name(name):
    pattern = "(RTX|GTX|GT|RX|WX|QUADRO|R7|R5|R9|HD|T)(.?.?)((\w*)\d+)( ?(TI|XT|X|SUPER))?"
    resume_name = re.search(pattern, name)
    if resume_name is None:
        return ''
    else:
        return resume_name.group().strip()

df['ds_nome_produto_resumido'] = df.apply(lambda x: get_resume_product_name(x['ds_nome_produto']), axis=1)

In [149]:
df.sample(10)

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data,ds_nome_produto_resumido
271715,Gigabyte,PLACA DE VIDEO GIGABYTE GEFORCE RTX 2060 SUPER...,2802.52,2802.52,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-15 22:01:40.276216+00:00,2022-11-15,RTX 2060 SUPER
316847,XFX,PLACA DE VIDEO XFX RADEON R9 4GB HBM NANO 4096...,3748.97,3748.97,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-24 10:01:25.317207+00:00,2022-11-24,R9 4
214921,EVGA,PLACA DE VIDEO EVGA GEFORCE GTX 1070 8GB GDDR5...,1873.84,1873.84,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-05 10:02:01.888203+00:00,2022-11-05,GTX 1070
256266,Galax,PLACA DE VIDEO GALAX GEFORCE RTX 3070 EX GAMER...,5795.37,5795.37,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-12 22:01:49.274075+00:00,2022-11-12,RTX 3070
267410,Afox,"PLACA DE VÍDEO AFOX GEFORCE GT730 4GB, DDR3, 1...",529.90,529.90,0.0,5,None,True,Kabum,2022-11-15 10:01:22.150237+00:00,2022-11-15,GT730
87251,Asus,PLACA DE VIDEO ASUS GEFORCE RTX 3080 10GB GDDR...,8636.25,8636.25,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-13 22:39:18.005597+00:00,2022-10-13,RTX 3080
268613,XFX,PLACA DE VIDEO XFX RADEON R7 250 2GB DDR3 LOW ...,385.22,385.22,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-15 10:01:22.150237+00:00,2022-11-15,R7 250
224430,Pny,PLACA DE VÍDEO PNY NVIDIA QUADRO T400 2GB GDDR...,1264.80,1264.80,0.0,1,None,True,Kabum,2022-11-07 10:04:11.114311+00:00,2022-11-07,QUADRO T400
333601,MSI,PLACA DE VIDEO MSI GEFORCE GTX 1660 SUPER 6GB ...,2272.69,2272.69,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-27 10:01:27.473085+00:00,2022-11-27,GTX 1660 SUPER
4479,AsRock,PLACA DE VIDEO ASROCK RADEON RX 6400 CHALLENGE...,1818.07,1420.34,12.0,<NA>,"[{'ds_oferta_nome': 'Quinzena do Cliente', 'vl...",IN_STOCK,Pichau,2022-09-29 22:01:30.406181+00:00,2022-09-29,RX 6400


# Transpostar para outra lugar depois

In [11]:
# Limpa o nome de arquivos RTX A2000, 12GB
import os
import re
folder = './teste/'
for filename in os.listdir('./teste'):
    nome = re.sub(r'\([^)]*\)', '', filename)
    nome = re.sub(r'\s+(?=[.,?!])', '', nome)
    os.rename(os.path.join(folder, filename), os.path.join(folder, nome))